In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

# Sklearn
from sklearn.model_selection import train_test_split

# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

%matplotlib inline

In [2]:
print("TensorFlow version: ", tf.__version__)
print("Tensorflow keras version: ", tf.keras.__version__)

TensorFlow version:  1.13.1
Tensorflow keras version:  2.2.4-tf


In [ ]:
# Load the datasets
data_text = x_train_text + x_test_text

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [ ]:
x_train_text[1]

In [ ]:
y_train[1]

### Tokenizer

In [ ]:
# We may instruct the tokenizer to only use 
# e.g. the 10000 most popular words from the data-set.
num_words = 10000

tokenizer = Tokenizer(num_words=num_words)

In [ ]:
%%time
tokenizer.fit_on_texts(data_text)

In [ ]:
if num_words is None:
    num_words = len(tokenizer.word_index)

In [ ]:
tokenizer.word_index

In [ ]:
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)

In [ ]:
x_train_text[1]

In [ ]:
np.array(x_train_tokens[1])

In [ ]:
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

### Padding and Truncating Data

In [ ]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [ ]:
np.mean(num_tokens)

In [ ]:
np.max(num_tokens)

In [ ]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

In [ ]:
# Calculate How much overs for the dataset
np.sum(num_tokens < max_tokens) / len(num_tokens)

In [ ]:
pad = 'pre'

x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

In [ ]:
x_train_pad.shape

In [ ]:
x_test_pad.shape

In [ ]:
np.array(x_train_tokens[1])

In [ ]:
x_train_pad[1]

### Tokenizer Inverse Map

In [ ]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [ ]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [ ]:
x_train_text[1]

In [ ]:
tokens_to_string(x_train_tokens[1])

### Create the Recurrent Neural Network

In [ ]:
model = Sequential()

In [ ]:
embedding_size = 8

In [ ]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

In [ ]:
model.add(GRU(units=16, return_sequences=True))

In [ ]:
model.add(GRU(units=8, return_sequences=True))

In [ ]:
model.add(GRU(units=4))

In [ ]:
model.add(Dense(1, activation='sigmoid'))

In [ ]:
optimizer = Adam(lr=1e-3)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
model.summary()

### Train the Recurrent Neural Network

In [ ]:
%%time
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=64)

### Example of Mis-Classified Text

In [ ]:
%%time
y_pred = model.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]

In [ ]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

In [ ]:
cls_true = np.array(y_test[0:1000])

In [ ]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [ ]:
len(incorrect)

In [ ]:
idx = incorrect[0]
idx

In [ ]:
text = x_test_text[idx]
text

In [ ]:
y_pred[idx]

In [ ]:
cls_true[idx]

### New Data

In [ ]:
text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Good movie!"
text3 = "Maybe I like this movie."
text4 = "Meh ..."
text5 = "If I were a drunk teenager then this movie might be good."
text6 = "Bad movie!"
text7 = "Not a good movie!"
text8 = "This movie really sucks! Can I get my money back please?"
texts = [text1, text2, text3, text4, text5, text6, text7, text8]

In [ ]:
tokens = tokenizer.texts_to_sequences(texts)

In [ ]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)
tokens_pad.shape

In [ ]:
model.predict(tokens_pad)

### Embeddings

In [ ]:
layer_embedding = model.get_layer('layer_embedding')

In [ ]:
weights_embedding = layer_embedding.get_weights()[0]

In [ ]:
weights_embedding.shape

In [ ]:
token_good = tokenizer.word_index['good']
token_good

In [ ]:
token_great = tokenizer.word_index['great']
token_great

In [ ]:
weights_embedding[token_good]

In [ ]:
weights_embedding[token_great]

In [ ]:
token_bad = tokenizer.word_index['bad']
token_horrible = tokenizer.word_index['horrible']

In [ ]:
weights_embedding[token_bad]

In [ ]:
weights_embedding[token_horrible]

### Sorted Words

In [ ]:
def print_sorted_words(word, metric='cosine'):
    """
    Print the words in the vocabulary sorted according to their
    embedding-distance to the given word.
    Different metrics can be used, e.g. 'cosine' or 'euclidean'.
    """

    # Get the token (i.e. integer ID) for the given word.
    token = tokenizer.word_index[word]

    # Get the embedding for the given word. Note that the
    # embedding-weight-matrix is indexed by the word-tokens
    # which are integer IDs.
    embedding = weights_embedding[token]

    # Calculate the distance between the embeddings for
    # this word and all other words in the vocabulary.
    distances = cdist(weights_embedding, [embedding],
                      metric=metric).T[0]
    
    # Get an index sorted according to the embedding-distances.
    # These are the tokens (integer IDs) for words in the vocabulary.
    sorted_index = np.argsort(distances)
    
    # Sort the embedding-distances.
    sorted_distances = distances[sorted_index]
    
    # Sort all the words in the vocabulary according to their
    # embedding-distance. This is a bit excessive because we
    # will only print the top and bottom words.
    sorted_words = [inverse_map[token] for token in sorted_index
                    if token != 0]

    # Helper-function for printing words and embedding-distances.
    def _print_words(words, distances):
        for word, distance in zip(words, distances):
            print("{0:.3f} - {1}".format(distance, word))

    # Number of words to print from the top and bottom of the list.
    k = 10

    print("Distance from '{0}':".format(word))

    # Print the words with smallest embedding-distance.
    _print_words(sorted_words[0:k], sorted_distances[0:k])

    print("...")

    # Print the words with highest embedding-distance.
    _print_words(sorted_words[-k:], sorted_distances[-k:])

In [ ]:
print_sorted_words('great', metric='cosine')

In [ ]:
print_sorted_words('worst', metric='cosine')